In [1]:
import anndata
import rds2py
import rds2py.read_matrix
import pandas as pd

In [2]:
def rds2Series(obj):
    val = obj['data']
    if ('class' in obj['attributes']) and (obj['attributes']['class'] == 'factor'):
        val = [obj['attributes']['levels']['data'][i-1] for i in val]
    val = pd.Series(val)
    if 'names' in obj['attributes']:
        val.index = obj['attributes']['names']['data']
    return val

def rds2DataFrame(obj):
    val = {obj['attributes']['names']['data'][i]:rds2Series(obj['data'][i]) for i in range(len(obj['attributes']['names']['data']))}
    val = pd.DataFrame(val)
    val.index = obj['attributes']['row.names']['data']
    return val

# Read rds

In [3]:
robj = rds2py.read_rds('rds/seurat.rds')

/software/cellgen/cellgeni/pm19/envs/sc/lib/python3.12/site-packages/rds2py/generics.py:137: RuntimeWarning: RDS file contains an unknown class: 'Seurat', returning the dictionary
  warn(


# Extract obs

In [6]:
obs = rds2DataFrame(robj['attributes']['meta.data'])
obs

,orig.ident,nCount_RNA,nFeature_RNA
AAACCTGCAGCCTGTG-1,1,5029.0,1819
AAACGGGTCGCTGATA-1,1,4343.0,1738
AAAGATGTCCTCGCAT-1,1,898.0,554
AAAGCAAAGTATGACA-1,1,4999.0,1796
AAAGCAATCCATTCTA-1,1,601.0,416
...,...,...,...
TTTGGTTTCAACGAAA-1,1,1408.0,815
TTTGGTTTCTGAGGGA-1,1,6074.0,2003
TTTGTCAAGCCCAATT-1,1,8067.0,2727
TTTGTCAAGTCTCAAC-1,1,4770.0,1767


# Extract var

In [7]:
gene_names = robj['attributes']['assays']['data'][0]['attributes']['features']['attributes']['dimnames']['data'][0]['data']
var = pd.DataFrame({'name':gene_names})
var.index = gene_names
var

,name
MIR1302-2HG,MIR1302-2HG
FAM138A,FAM138A
OR4F5,OR4F5
AL627309.1,AL627309.1
AL627309.3,AL627309.3
...,...
AC141272.1,AC141272.1
AC023491.2,AC023491.2
AC007325.1,AC007325.1
AC007325.4,AC007325.4


# Extract counts

In [8]:
X = rds2py.read_matrix._as_sparse_matrix(robj['attributes']['assays']['data'][0]['attributes']['layers']['data'][0]).T
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1798509 stored elements and shape (972, 36601)>

In [9]:
adata = anndata.AnnData(X=X,obs=obs,var=var)
adata

AnnData object with n_obs × n_vars = 972 × 36601
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA'
    var: 'name'

In [10]:
adata.write_h5ad('test.h5ad')